In [1]:
import datetime
import math

import pandas as pd
import numpy as np

from IPython.display import display
from scipy.optimize import root_scalar, RootResults

from functions import RatesUtil
from functions.assetClasses.bond import BondPricing

Question 4.1 solution

In [ ]:
print(RatesUtil.conversion_btw_discrete_continuous(0.09, 4, True))
print(RatesUtil.conversion_btw_discrete(0.14, 4, 1))

Question 4.3 Solution

In [ ]:
bondPrice: float = BondPricing.bond_pricing_vanilla(100, 1.5, 0.08, 0.104)
print(f"Bond Price = {bondPrice}")

# The second part of the question could be solved using and adhoc method but lack extensibility, here we go
# For 6 month and 1-year coupon payment the present value is
present_value: float = 4/1.05 + 4/(1.05**2)
print(f"6 and 12 Month present value = {present_value}")

diff: float = bondPrice - present_value
diff = 104*(1/diff)
zeros_18: float = 2*(diff**(1/3) -1)
print(f"18 Month zero rate = {zeros_18}")

# The better approach is to use either Newton-Raphson, bisection, secant method to solve the problem
# this method is more production like code

# define your function
def finding_zeros(zero_18: float) -> float:
    return present_value + 104/(1+zero_18/2)**3 - bondPrice


result: RootResults = root_scalar(finding_zeros, method='secant', x0=0.001, x1=1)
print(result)

payout: float = 1100.00
initial_investment: float = 1000.00
annual: float = RatesUtil.get_percentage_return(initial_investment, payout, 1.0, 1.0)
semi_annual: float = RatesUtil.get_percentage_return(initial_investment, payout, 1, 2)
monthly: float = RatesUtil.get_percentage_return(initial_investment, payout, 1, 12)
continuos: float = RatesUtil.get_percentage_return(initial_investment, payout, is_continuous=True)

print(f"Given payout of {payout} and initial investment of {initial_investment} Annual compounding is {annual}")
print(f"Given payout of {payout} and initial investment of {initial_investment} Semiannual compounding is {semi_annual}")
print(f"Given payout of {payout} and initial investment of {initial_investment} Monthly compounding is {monthly}")
print(f"Given payout of {payout} and initial investment of {initial_investment} continuous compounding is {continuos}")

Question 4_5 Solution

In [ ]:
zeros_test_df: pd.DataFrame = pd.DataFrame({'Maturity': [3, 6, 9, 12, 15, 18],
                                            'Zero_Rate': [8.0, 8.2, 8.4, 8.5, 8.6, 8.7]})
display(zeros_test_df)
display(BondPricing.forward_rates(zeros_test_df))

The Solution to Question 4.6 follows from the answer from Question 4.5
Based on the ending time of between 1 year and 1.25 years is choosen as the period, therefore

$R_{2}$ = 8.6% <br>
$R_{1}$ = 8.5% <br>
$T_{2}$ = 15.0 (1.25 years) <br>
$T_{1}$ = 12.0 (1.00 year) <br>

$R_{F}$ = 9.0% (Continuous compounding) <br>

In [ ]:
# convert the forward rate from continuous compounding to quarterly compounding
forward_rate: float = RatesUtil.conversion_btw_discrete_continuous(0.09, 4, True)
print(forward_rate)

pay_off:float = 1000000*(0.095 - forward_rate)*(1.25 - 1.00) * math.exp(-0.086*1.25)
print(pay_off)

In [9]:
from functions.daysCount import actual_actual, thirty_three_sixty
from datetime import date

settle_date: date = date(2018, 3, 5)
coupon_dates: list = [date(2018, 1, 10), date(2018, 7, 10)]

print( actual_actual(settle_date, coupon_dates) )
print( thirty_three_sixty(settle_date, coupon_dates) )

(54, 181, 0.2983425414364641)
(51, 180, 0.2833333333333333)


Solution for Question 4_11 Hull Seventh Edition

In [6]:

df = pd.DataFrame({'maturities': [0.5, 1.0, 1.5, 2.0, 2.5], 'zero_rates': [0.04, 0.042, 0.044, 0.046, 0.048],
                   'payments':[2,2,2,2,102]})
p: float = BondPricing.bond_pricing(df)
print(df)
print(p)

   maturities  zero_rates  payments
0         0.5       0.040         2
1         1.0       0.042         2
2         1.5       0.044         2
3         2.0       0.046         2
4         2.5       0.048       102
98.04049348058196


Solution for Question 4_13

In [39]:
df = pd.DataFrame({'maturities': [0.5, 1.0, 1.5, 2.0], 'zero_rates': [0.05, 0.06, 0.065, 0.07], 'payments':[2,2,2,102]})
cash_price: float = 100.0

#Zero function used by the root finder
#
def bond_pricing_func(coupon_val: float, price=cash_price, zeros_df: pd.DataFrame = df) -> float :

    temp_list: list = [ coupon_val*x for x in np.ones(4).tolist() ]
    temp_list[-1] = temp_list[-1] + 100

    df['payments'] = pd.Series(temp_list)

    return price - BondPricing.bond_pricing(zeros_df)

display(df)

result: RootResults = root_scalar(bond_pricing_func, method='secant', x0=0.001, x1=10)

print(result)
display(df)


,maturities,zero_rates,payments
0,0.5,0.050,2
1,1.0,0.060,2
2,1.5,0.065,2
3,2.0,0.070,102


      converged: True
           flag: converged
 function_calls: 3
     iterations: 2
           root: 3.5370387393915
         method: secant


,maturities,zero_rates,payments
0,0.5,0.050,3.537039
1,1.0,0.060,3.537039
2,1.5,0.065,3.537039
3,2.0,0.070,103.537039


Solution to Question 4_14

In [3]:
zeros_test_df: pd.DataFrame = pd.DataFrame({'Maturity': [1, 2, 3, 4, 5],
                                            'Zero_Rate': [2.0, 3.0, 3.7, 4.2, 4.5]})
display(zeros_test_df)
display(BondPricing.forward_rates(zeros_test_df))

,Maturity,Zero_Rate
0,1,2.0
1,2,3.0
2,3,3.7
3,4,4.2
4,5,4.5


,Maturity,Zero Rates,Forward Rate
0,1.0,2.0,0.0
1,2.0,3.0,4.0
2,3.0,3.7,5.1
3,4.0,4.2,5.7
4,5.0,4.5,5.7


Question:

An investor buys a 5% callable corporate bond at 95 with 20 years until maturity. The bond was called five years later at 105. What is the YTC (Yield to Call)

The approximation formula I found on the internet was

$YTC = \frac{2*(Annual Interest + Annual Accretion)}{Call\,price + Market\,price}$

$Annual Accretion = \frac{Call\,Price - Market\,Prices}{Number\,Of\,Years\,to\,Call}$

But if you want to really create a production quality code, then one should use any of the numeric methods for root find and replace the face value of the bond with the call price and find r which the YTC


In [17]:
import scipy.optimize as opt

annual_interest: float = 0.05 * 1000
call_price:float = 105*10
market_price:float = 95*10
annual_accretion: float = (call_price - market_price)/5
ytc: float = (2*(annual_interest + annual_accretion))/(call_price + market_price)
print(ytc)

# A better solution

def root_function(r: float) -> float:
    return market_price - BondPricing.bond_pricing_vanilla(call_price, 5, 0.05,r, annual_interest/2)

# Secant method for faster convergence but could follow wrong path
result: RootResults = root_scalar(root_function, method='secant', x0=0.01, x1=1)
print(result)

#Slower convergence but always accurate
ytm = opt.bisect(root_function, 0.001, 10, full_output= True)
print(ytm)


0.07
      converged: True
           flag: converged
 function_calls: 13
     iterations: 12
           root: 0.07055416276653798
         method: secant
(0.07055416276727658,       converged: True
           flag: converged
 function_calls: 45
     iterations: 43
           root: 0.07055416276727658
         method: bisect)


In [2]:
def temp_func(x: float) -> float:
    return 2**x - x**32

result: RootResults = root_scalar(temp_func, method='secant', x0=0.0, x1=1000)
print(result)

      converged: True
           flag: converged
 function_calls: 3
     iterations: 2
           root: -1.8665272370064378e-298
         method: secant
